In [1]:
import sympy
sympy.init_printing()

In [2]:
R, r, t, dr, dt = sympy.symbols('R, r, \\theta, \\delta{r}, \\delta{\\theta}')
R, r, t, dr, dt

(R, r, \theta, \delta{r}, \delta{\theta})

In [3]:
import sympy
sympy.init_printing()

In [4]:
R, r, t, dr, dt = sympy.symbols('R, r, \\theta, \\delta{r}, \\delta{\\theta}')
R, r, t, dr, dt

(R, r, \theta, \delta{r}, \delta{\theta})

In [33]:
def genExpress(r, t, R):
    exp = sympy.sqrt(1 + (r / R)**2)
    rhat = 2 * R**2 * r / (R**2 + r**2)
    return rhat * sympy.cos(t), rhat * sympy.sin(t), sympy.sqrt(R**2 - rhat**2)

In [34]:
x1, y1, z1 = genExpress(r, t, R)
x2, y2, z2 = genExpress(r + dr, t, R)
x3, y3, z3 = genExpress(r + dr, t + dt, R)
x1, y1, z1, x2, y2, z2, x3, y3, z3

⎛                                               ___________________           
⎜   2                   2                      ╱       4  2             2     
⎜2⋅R ⋅r⋅cos(\theta)  2⋅R ⋅r⋅sin(\theta)       ╱     4⋅R ⋅r       2   2⋅R ⋅(\de
⎜──────────────────, ──────────────────,     ╱   - ────────── + R  , ─────────
⎜      2    2              2    2           ╱               2              2  
⎜     R  + r              R  + r           ╱       ⎛ 2    2⎞              R  +
⎝                                        ╲╱        ⎝R  + r ⎠                  

                                                                  ____________
                            2                                    ╱       4    
lta{r} + r)⋅cos(\theta)  2⋅R ⋅(\delta{r} + r)⋅sin(\theta)       ╱     4⋅R ⋅(\d
───────────────────────, ────────────────────────────────,     ╱   - ─────────
                2              2                  2           ╱               
 (\delta{r} + r)              R  + (\delta{r} + r) 

In [35]:
ds2 = (x3 - x1)**2 + (y3 - y1)**2 + (z3 - z1)**2
ds1_2 = (x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2
ds2_2 = (x3 - x2)**2 + (y3 - y2)**2 + (z3 - z2)**2
# E = ds1_2.expand() + ds2_2.expand() - ds2.expand()
# E
ds1_2

                                                                              
                                                         2                    
⎛     2                    2                            ⎞    ⎛     2          
⎜  2⋅R ⋅r⋅sin(\theta)   2⋅R ⋅(\delta{r} + r)⋅sin(\theta)⎟    ⎜  2⋅R ⋅r⋅cos(\th
⎜- ────────────────── + ────────────────────────────────⎟  + ⎜- ──────────────
⎜        2    2               2                  2      ⎟    ⎜        2    2  
⎝       R  + r               R  + (\delta{r} + r)       ⎠    ⎝       R  + r   
                                                                              

                                                                              
                                        2   ⎛         ___________________     
          2                            ⎞    ⎜        ╱       4  2             
eta)   2⋅R ⋅(\delta{r} + r)⋅cos(\theta)⎟    ⎜       ╱     4⋅R ⋅r       2      
──── + ────────────────────────────────⎟  + ⎜-     

In [36]:
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

In [38]:
deg = 2
ll = Taylor_polynomial_sympy(function_expression=ds1_2.expand(), # 计算 ds^2
                             variable_list=[dr, dt],
                             evaluation_point=[0, 0],
                             degree=deg)
ll.simplify()

    4          2 
 4⋅R ⋅\delta{r}  
─────────────────
 4      2  2    4
R  + 2⋅R ⋅r  + r 